# Выбор локации для скважины

Допустим, мы работаем в добывающей компании, для которой нужно решить, где бурить новую скважину.

Нам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Построим модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируем возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

Загрузим библиотеки

In [1]:
import pandas as pd
!pip3 install pandas-profiling==2.11
from pandas_profiling import ProfileReport
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from mport numpy as npsklearn.model_selection import RandomizedSearchCV
i
from scipy.stats import randint
from sklearn.metrics import mean_squared_error
from numpy.random import RandomState
from scipy import stats as st
import warnings
warnings.filterwarnings("ignore")

In [2]:
RANDOM_STATE = 12345
BUDGET = 10_000_000_000
PRICE_PER_BAREL = 450
INCOME_PER_PRODUCT = 450000

Откроем по отдельности файлы, посмотрим общую информацию и часть датасета, остальные данные выведем с помощью инструмента ProfileReport, сделаем вывод по каждому региону отдельно

In [3]:
try:
    data_0 = pd.read_csv('/datasets/geo_data_0.csv')
except:
    data_0 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_0.csv')

In [4]:
data_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [5]:
data_0.head(5)

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [6]:
profile = ProfileReport(data_0,
                        title='Pandas Profiling Report - Data 0')

profile.to_widgets()

Summarize dataset:   0%|          | 0/18 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

Вывод: открыли и рассмотрели данные по первому региону (0), пропусков нет, дубликатов нет, распределения по точкам разные, есть нормальное и ненормальное распределение

In [7]:
try:
    data_1 = pd.read_csv('/datasets/geo_data_1.csv')
except:
    data_1 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_1.csv')

In [8]:
data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [9]:
data_1.head(5)

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [10]:
profile = ProfileReport(data_1,
                        title='Pandas Profiling Report - Data 1')

profile.to_widgets()

Summarize dataset:   0%|          | 0/18 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

Вывод: открыли и рассмотрели данные по второму региону (1), пропусков нет, дубликатов нет, распределения точек также имеют разное распределение и нормальное и ненормальное, с данными первого региона не совпадают. Есть связь между продуктом и точкой f2.

In [11]:
try:
    data_2 = pd.read_csv('/datasets/geo_data_2.csv')
except:
    data_2 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_2.csv')

In [12]:
data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [13]:
data_2.head(5)

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [14]:
profile = ProfileReport(data_2,
                        title='Pandas Profiling Report - Data 2')

profile.to_widgets()

Summarize dataset:   0%|          | 0/18 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

Вывод: открыли и рассмотрели данные по второму региону (2), пропусков нет, дубликатов нет, распределения всех точек имеют нормальное распределение.

Во всех датасетах есть повторяющиеся id, посмотрим разные ли там данные на примере одного региона, учитывая, что дубликатов нет, разные данные, но перепроверим

In [15]:
data_2['id'].value_counts()

xCHr8    2
Vcm5J    2
KUPhW    2
VF7Jo    2
9CcOt    1
        ..
6Xn7I    1
2dMsg    1
qO7bn    1
3L9g4    1
LlFB5    1
Name: id, Length: 99996, dtype: int64

In [16]:
data_2.query('id == "xCHr8"')

,id,f0,f1,f2,product
28039,xCHr8,1.633027,0.368135,-2.378367,6.120525
43233,xCHr8,-0.847066,2.101796,5.597130,184.388641


данные разные, удалим во всех датасетах колонки с id, чтобы они не мешали нам в моделировании

In [17]:
data_0 = data_0.drop(['id'], axis=1)
data_1 = data_1.drop(['id'], axis=1)
data_2 = data_2.drop(['id'], axis=1)

Проверим

In [18]:
data_0.head(5)

,f0,f1,f2,product
0,0.705745,-0.497823,1.221170,105.280062
1,1.334711,-0.340164,4.365080,73.037750
2,1.022732,0.151990,1.419926,85.265647
3,-0.032172,0.139033,2.978566,168.620776
4,1.988431,0.155413,4.751769,154.036647


Вывод: открыли и рассмотрели данные по всем регионам, удалили несущественные данные, посмотрели, что нет дубликатов и пропусков, посмотрели по графикам какие есть распределения.

## Обучение и проверка модели

Обучим и проверим модели для каждого региона:
 1. Разобьем данные на обучающую и валидационную выборки в соотношении 75:25.
 2. Обучим модель и сделаем предсказания на валидационной выборке.
 3. Сохраним предсказания и правильные ответы на валидационной выборке.
 4. Напечатаем на экране средний запас предсказанного сырья и RMSE модели.
 5. Проанализируем результаты.

Первый регион

In [19]:
def shape(features_train, target_train, features_valid, target_valid):
    print(
    'Обучающая выборка:', features_train_0.shape, target_train_0.shape,
    'Валидационная выборка:', features_valid_0.shape, target_valid_0.shape
     )

In [20]:
def parameters(features_train, target_train):
    rs_space={'fit_intercept':[True, False],
              'normalize':[True, False],
              'copy_X':[True, False]
         }
    lr = LinearRegression()
    lr_random = RandomizedSearchCV(lr, rs_space, n_iter=500, scoring='neg_mean_squared_error', n_jobs=-1, cv=3)
    model_random = lr_random.fit(features_train, target_train)
    print('Best hyperparameters are: '+str(model_random.best_params_))
    print('Best score is: '+str(model_random.best_score_))

In [21]:
def RMSE(mse):
    print("RMSE: {}".format(np.sqrt(mse)))

In [22]:
features_0 = data_0.drop(['product'], axis=1)
target_0 = data_0['product']

features_train_0, features_valid_0, target_train_0, target_valid_0  = train_test_split(
    features_0, target_0, test_size=0.25, random_state=12345) 

In [23]:
shape(features_train_0, target_train_0, features_valid_0, target_valid_0)

Обучающая выборка: (75000, 3) (75000,) Валидационная выборка: (25000, 3) (25000,)


Сделаем подбор лучних параметров через случайный перебор для каждого региона

In [24]:
parameters(features_train_0, target_train_0)

Best hyperparameters are: {'normalize': False, 'fit_intercept': True, 'copy_X': True}
Best score is: -1423.8028045400797


In [25]:
model_0 = LinearRegression(normalize=True, fit_intercept=True, copy_X=True)
model_0.fit(features_train_0, target_train_0)
predictions_valid_0 = model_0.predict(features_valid_0)
mse = mean_squared_error(target_valid_0, predictions_valid_0) 
RMSE(mse)
print(f'Средний запас сырья: {predictions_valid_0.mean()}.')

RMSE: 37.5794217150813
Средний запас сырья: 92.59256778438035.


Второй регион

In [26]:
features_1 = data_1.drop(['product'], axis=1)
target_1 = data_1['product']

features_valid_1, features_train_1, target_valid_1, target_train_1 = train_test_split(
    features_1, target_1, test_size=0.75, random_state=12345)

shape(features_valid_1, features_train_1, target_valid_1, target_train_1)

Обучающая выборка: (75000, 3) (75000,) Валидационная выборка: (25000, 3) (25000,)


In [27]:
parameters(features_train_1, target_train_1)

Best hyperparameters are: {'normalize': False, 'fit_intercept': True, 'copy_X': True}
Best score is: -0.7946568195931537


In [28]:
model_1 = LinearRegression(normalize=False, fit_intercept=True, copy_X=True)
model_1.fit(features_train_1, target_train_1)
predictions_valid_1 = model_1.predict(features_valid_1)
mse = mean_squared_error(target_valid_1, predictions_valid_1) 
RMSE(mse)
print(f'Средний запас сырья: {predictions_valid_1.mean()}.')

RMSE: 0.8873287354658539
Средний запас сырья: 68.8598208262229.


Третий регион

In [29]:
features_2 = data_2.drop(['product'], axis=1)
target_2 = data_2['product']

features_valid_2, features_train_2, target_valid_2, target_train_2 = train_test_split(
    features_2, target_2, test_size=0.75, random_state=12345)

shape(features_valid_2, features_train_2, target_valid_2, target_train_2)

Обучающая выборка: (75000, 3) (75000,) Валидационная выборка: (25000, 3) (25000,)


In [30]:
parameters(features_train_2, target_train_2)

Best hyperparameters are: {'normalize': True, 'fit_intercept': True, 'copy_X': True}
Best score is: -1603.167409014255


In [31]:
model_2 = LinearRegression(normalize=True, fit_intercept=True, copy_X=True)
model_2.fit(features_train_2, target_train_2)
predictions_valid_2 = model_2.predict(features_valid_2)
mse = mean_squared_error(target_valid_2, predictions_valid_2) 
RMSE(mse)
print(f'Средний запас сырья: {predictions_valid_2.mean()}.')

RMSE: 40.11167877627781
Средний запас сырья: 95.06093851120131.


Вывод: наименьший показатель RMSE во втором регионе, там есть корреляция между продуктом и точкой f2, получается по предсказаниям самый лучший регион - это второй регион. 

## Подготовка к расчёту прибыли

Подготовимся к расчёту прибыли:
 1. Все ключевые значения для расчётов сохраним в отдельных переменных.
 2. Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины. Сравним полученный объём сырья со средним запасом в каждом регионе. 
 3. Напишем выводы по этапу подготовки расчёта прибыли.

Бюджет на разработку скважин в регионе — 10 млрд рублей.
При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.

In [32]:
product_0 = data_0['product']
product_1 = data_1['product']
product_2 = data_2['product']
def earnings(product, income, budget):
    return (product.sum() * income) - budget
volume_without_loss = round((BUDGET/INCOME_PER_PRODUCT/200), 2)
print(f'Объём сырья для безубыточной разработки новой скважины:{volume_without_loss}')
def mean_product(product):
    return round(product.mean(),2)
print(f'Средний объем - первый регион: {mean_product(product_0)}, средний объем - второй регион: {mean_product(product_1)}, средний объем - третий регион: {mean_product(product_2)}.')

Объём сырья для безубыточной разработки новой скважины:111.11
Средний объем - первый регион: 92.5, средний объем - второй регион: 68.83, средний объем - третий регион: 95.0.


Вывод: 
1. создали переменные для продукта по каждому региону 
2. написали формулу для расчета прибыли по региону, в которой количество продукта региона умножили на его стоимость и вычли бюджет на разработку
3. посчитали объем сырья для одной скважины для безубыточной разработки: поделили бюджет на доход от единицы продукта и поделили на 200 скважин, которые будут разрабатываться
4. посмотрели средние объемы скважин по регионам, средние объемы меньше, чем нужный показатель.

## Расчёт прибыли и рисков 

Напишем функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели:
 1. Выберем скважины с максимальными значениями предсказаний. 
 2. Просуммируем целевое значение объёма сырья, соответствующее этим предсказаниям.
 3. Рассчитаем прибыль для полученного объёма сырья.

Создадим датафреймы с нужными данными для каждого региона, потом напишем формулу для расчета прибыли по 200 лучшим скважинам по предсказаниям, посчитанным по реальным данным

In [33]:
data_region_0 = pd.DataFrame()
data_region_0['target_valid_0'] = target_valid_0
data_region_0['predictions_valid_0'] = predictions_valid_0.tolist()
data_region_0.reset_index(inplace=True)
data_region_0.drop(['index'], axis=1, inplace=True)
data_region_0.head(5)

,target_valid_0,predictions_valid_0
0,10.038645,95.894952
1,114.551489,77.572583
2,132.603635,77.892640
3,169.072125,90.175134
4,122.325180,70.510088


In [34]:
data_region_1 = pd.DataFrame()
data_region_1['target_valid_1'] = target_valid_1
data_region_1['predictions_valid_1'] = predictions_valid_1.tolist()
data_region_1.reset_index(inplace=True)
data_region_1.drop(['index'], axis=1, inplace=True)
data_region_1.head(5)

,target_valid_1,predictions_valid_1
0,0.000000,1.041699
1,26.953261,26.737106
2,3.179103,2.427588
3,53.906522,55.368477
4,137.945408,137.031307


In [35]:
data_region_2 = pd.DataFrame()
data_region_2['target_valid_2'] = target_valid_2
data_region_2['predictions_valid_2'] = predictions_valid_2.tolist()
data_region_2.reset_index(inplace=True)
data_region_2.drop(['index'], axis=1, inplace=True)
data_region_2.head(5)

,target_valid_2,predictions_valid_2
0,96.153932,103.575400
1,15.864224,56.774739
2,55.397694,105.810890
3,126.129059,117.868015
4,5.334678,103.626441


In [36]:
def revenue(target, probabilities):
    probs_sorted = probabilities.sort_values(ascending=False) 
    selected = target[probs_sorted.index][:200]
    return selected.sum()*INCOME_PER_PRODUCT - BUDGET
   

result_0 = revenue(data_region_0['target_valid_0'], data_region_0['predictions_valid_0'])
result_1 = revenue(data_region_1['target_valid_1'], data_region_1['predictions_valid_1'])
result_2 = revenue(data_region_2['target_valid_2'], data_region_2['predictions_valid_2'])

print(f'Прибыль первого региона: {result_0}, прибыль второго региона: {result_1}, прибыль третьего региона {result_2}.')

Прибыль первого региона: 3320826043.1398506, прибыль второго региона: 2415086696.681511, прибыль третьего региона 2684845723.912676.


Посчитаем риски и прибыль для каждого региона:

1. Применим технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.
2. Найдем среднюю прибыль, 95%-й доверительный интервал и риск убытков. Убыток — это отрицательная прибыль.
3. Напишем выводы: предложим регион для разработки скважин и обоснуем выбор.


Первый регион

In [37]:
state = np.random.RandomState(12345)


values = []
for i in range(1000):
    target_subsample = data_region_0['target_valid_0'].sample(n=500, replace=True, random_state=state)
    probs_subsample = data_region_0['predictions_valid_0'][target_subsample.index]
    subsample = revenue(target_subsample, probs_subsample)
    values.append(subsample) 

values = pd.Series(values)

mean = values.mean()
print(f'Средняя выручка: {mean}.')
print(f'95%-ый доверительный интервал c {values.quantile(0.025)} до {values.quantile(0.975)}.')
print('Риск убытков = {:.2%} '.format((pd.Series(values)<0).mean()))

Средняя выручка: 425938526.91059244.
95%-ый доверительный интервал c -102090094.83793654 до 947976353.3583689.
Риск убытков = 6.00% 


Второй регион

In [39]:
state = np.random.RandomState(12345)


values = []
for i in range(1000):
    target_subsample = data_region_1['target_valid_1'].sample(n=500, replace=True, random_state=state)
    probs_subsample = data_region_1['predictions_valid_1'][target_subsample.index]
    subsample = revenue(target_subsample, probs_subsample)
    values.append(subsample) 

values = pd.Series(values)

mean = values.mean()
print(f'Средняя выручка: {mean}.')
print(f'95%-ый доверительный интервал c {values.quantile(0.025)} до {values.quantile(0.975)}.')
print('Риск убытков = {:.2%} '.format((pd.Series(values)<0).mean()))

Средняя выручка: 491757264.3892312.
95%-ый доверительный интервал c 81624191.16619082 до 925837646.5739056.
Риск убытков = 0.90% 


Третий регион

In [40]:
state = np.random.RandomState(12345)


values = []
for i in range(1000):
    target_subsample = data_region_2['target_valid_2'].sample(n=500, replace=True, random_state=state)
    probs_subsample = data_region_2['predictions_valid_2'][target_subsample.index]
    subsample = revenue(target_subsample, probs_subsample)
    values.append(subsample) 

values = pd.Series(values)

mean = values.mean()
print(f'Средняя выручка: {mean}.')
print(f'95%-ый доверительный интервал c {values.quantile(0.025)} до {values.quantile(0.975)}.')
print('Риск убытков = {:.2%} '.format((pd.Series(values)<0).mean()))

Средняя выручка: 355594873.3653203.
95%-ый доверительный интервал c -223696673.3691303 до 942181246.7649084.
Риск убытков = 9.50% 


Вывод: второй регион выбираем как перспективный, так как он наилучший по предсказаниям, в этом регионе в единственном риск убытков меньше 2.5%, средняя прибыль в этом регионе выше других. 